In [41]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from io import StringIO
from datetime import datetime

In [58]:
def extract(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    table = soup.find('span', string="By market capitalization").find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
  
    return df
extract("https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks")

,Rank,Bank name,Market cap (US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [59]:
def transform(df, csvPath):
    # CSV se exchange rates read karna
    exchange_df = pd.read_csv(csvPath, index_col=0)
    print("Exchange CSV columns:", exchange_df.columns.tolist())
    # Try to find the correct column for rates
    if 'Rate' in exchange_df.columns:
        exchange_rate = exchange_df['Rate'].to_dict()
    else:
        # If not found, use the first column as rate
        exchange_rate = exchange_df[exchange_df.columns[0]].to_dict()
    print("Exchange Rates:", exchange_rate)
    
    # Column names ko ensure karna
    if 'Market cap (US$ billion)' in df.columns:
        column_name = 'Market cap (US$ billion)'
    else:
        raise ValueError("Market cap column not found in DataFrame")
    
    # Naye columns add karna with correct keys
    df['Market cap (PKR Billion)'] = df[column_name] * exchange_rate.get('PKR', 1)
    df['Market cap (INR Billion)'] = df[column_name] * exchange_rate.get('INR', 1)
    df['Market cap (UAE Dirham Billion)'] = df[column_name] * exchange_rate.get('UAE Dirham', 1)
    df['Market cap (Pound Billion)'] = df[column_name] * exchange_rate.get('Pound', 1)
    
    return df

# Main execution
df = extract("https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks")
csvPath = './input/exchange.csv'
transformed_df = transform(df, csvPath)

# Save and display transformed DataFrame
transformed_df.to_csv('./input/exchange.csv', index=False)
print("Transformed Data:\n", transformed_df.head())


Exchange CSV columns: ['Country_name', 'price']
Exchange Rates: {1: 'pk', 2: 'china'}
Transformed Data:
    Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   

   Market cap (PKR Billion)  Market cap (INR Billion)  \
0                    432.92                    432.92   
1                    231.52                    231.52   
2                    194.56                    194.56   
3                    160.68                    160.68   
4                    157.91                    157.91   

   Market cap (UAE Dirham Billion)  Market cap (Pound Billion)  
0                 